Show machine information for reproducibility:

In [ ]:
versioninfo()

Activate environment:

In [ ]:
using Pkg
Pkg.activate(pwd())
Pkg.instantiate()
# # add packages if needed
# Pkg.add(["BenchmarkTools"])
Pkg.status()

In [ ]:
# Load packages
using BenchmarkTools, LinearAlgebra, Random, Statistics

## Q1. Git/GitHub

**No handwritten homework reports are accepted for this course.**  We work with Git/GitHub.  Efficient and abundant use of Git, e.g., **frequent and well-documented** commits, is an important criterion for grading your homework.

1. If you don't have a GitHub account, apply for the [Student Developer Pack](https://education.github.com/pack) at GitHub using your UCLA email.

2. Create a **private** repository `biostat-m257-2025-spring` and add `Hua-Zhou` and `BrendonChau` (TA) as your collaborators.

3. Top directories of the repository should be `hw1`, `hw2`, ... You may create other branches for developing your homework solutions; but the `master` branch will be your presentation area. Put your homework submission files (Jupyter notebook `.ipynb`, html converted from notebook, `Project.toml`, all code and data set to reproduce results) in the `master` branch. 

4. After each homework due date, teaching assistant and instructor will check out your `master` branch for grading. Tag each of your homework submissions with tag names `hw1`, `hw2`, ...  Tagging time will be your submission time. That means if you tag your hw1 submission after deadline, penalty points will be deducted for late submission.  

5. Read the [style guide](https://github.com/invenia/BlueStyle) for Julia programming. Following rules in the style guide will be strictly enforced when grading: (1) four space indenting rule, (2) 92 charcter rule, (3) space after comma rule, (4) no space before comma rule, (5) space around binary operator rule.

## Q2. Computer arithmetics (18 points)

Let's check whether floating-point numbers obey certain algebraic rules. If a rule does not hold, one counter-example suffices. If a rule holds, briefly explain why it holds.

1. Associative rule for addition says `(x + y) + z == x + (y + z)`. Check association rule using `x = 0.1`, `y = 0.1` and `z = 1.0` in Julia. Explain what you find.

2. Do floating-point numbers obey the associative rule for multiplication: `(x * y) * z == x * (y * z)`?

3. Do floating-point numbers obey the distributive rule: `a * (x + y) == a * x + a * y`?  

4. Is `0 * x == 0` true for all floating-point number `x`? 

5. Is `x / a == x * (1 / a)` always true?

6. The commutative law for addition says `x + y = y + x` and the commutative law for multiplication says `x * y = y * x`. Do these two communtative laws hold for floating-point numbers? 

## Q3. Multiple dispatch and JIT (18 points)

1. Consider Julia function
```julia
function g(k)
    for i in 1:10
        k = 5k - 1
    end
    k
end
```
a. Use `@code_llvm` to find the LLVM bitcode of compiled `g` with `Int64` input.   
b. Use `@code_llvm` to find the LLVM bitcode of compiled `g` with `Float64` input.  
c. Compare the bitcode from questions 1 and 2. Explain what do you find.  
d. Read Julia documentation on `@fastmath` and repeat the questions 1-3 on the function  

```julia
function g_fastmath(k)  
    @fastmath for i in 1:10  
        k = 5k - 1
    end
    k
end
```
e. Explain what does the macro `@fastmath` do? And why are the bitcodes for `g` and `g_fastmath` with `Float64` input different? (Hint: Q2)

2. Benchmark the following two codes
```julia
x = 1:2:10^8
@benchmark sum($x)
```
and
```julia
y = collect(1:2:10^8)
@benchmark sum($y)
```
Explain the difference in performance.

## Q4. Sample variance (20 points)

Given a vector of numbers $x_1, \ldots, x_n$, we want to calculate the sample variance.

1. Implement the textbook algorithm for sample variance
$$
    s^2 = \frac{1}{n - 1} \sum_{i=1}^n (x_i - \bar{x})^2.
$$
How many floating-point operations are needed to compute the sample variance uising this textbook (two-pass) algorithm?

2. Implement the one-pass algorithm for sample variance
$$
    s^2 = \frac{1}{n - 1} \left( \sum_{i=1}^n x_i^2 - n \bar{x}^2 \right).
$$
How many floating-point operations are needed to compute the sample variance using this one-pass algorithm?

3. Generate data by
```julia
Random.seed!(257)
x = rand(1_000_000) .+ 1e8;
```
and benchmark the two-pass and one-pass algorithms. Memory allocations should be zero. 

4. Display the results of the two-pass and one-pass algorithms on this test data. Any surprises? Explain what you found.

5. Investigate the algorithm implemented in the Julia function `var`. Is the `var` function the textbook (two-pass) algorithm or the one-pass algorithm? Display the result on the test data and benchmark it. Explain what you found.

## Q5. Woodbury formula (12 points)

Demonstrate the following results in Julia (one numerical example for each fact). Mathematically curious ones are encouraged to prove them. 

1. **Sherman-Morrison formula**:
$$
	(\mathbf{A} + \mathbf{u} \mathbf{u}^T)^{-1} = \mathbf{A}^{-1} - \frac{1}{1 + \mathbf{u}^T \mathbf{A}^{-1} \mathbf{u}} \mathbf{A}^{-1} \mathbf{u} \mathbf{u}^T \mathbf{A}^{-1},
$$
where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is nonsingular and $\mathbf{u} \in \mathbb{R}^n$. This formula supplies the inverse of the symmetric, rank-one  perturbation of $\mathbf{A}$.

2. **Woodbury formula**:
$$
	(\mathbf{A} + \mathbf{U} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1},
$$
where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is nonsingular, $\mathbf{U}, \mathbf{V} \in \mathbb{R}^{n \times m}$, and $\mathbf{I}_m$ is the $m \times m$ identity matrix. In many applications $m$ is much smaller than $n$. Woodbury formula generalizes Sherman-Morrison and is valuable because the smaller matrix $\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U}$ is cheaper to invert than the larger matrix $\mathbf{A} + \mathbf{U} \mathbf{V}^T$.

3. **Binomial inversion formula**:
$$
	(\mathbf{A} + \mathbf{U} \mathbf{B} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1},
$$
where $\mathbf{A}$ and $\mathbf{B}$ are nonsingular.

4. **Determinant identity**:
$$
	\text{det}(\mathbf{A} + \mathbf{U} \mathbf{V}^T) = \text{det}(\mathbf{A}) \text{det}(\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U}).
$$
This formula is useful for evaluating the density of a multivariate normal with covariance matrix $\mathbf{A} + \mathbf{U} \mathbf{V}^T$.

## Q6. Triangular matrix and orthogonal matrix (18 points)

Demonstrate the following facts about triangular and orthogonal matrices in Julia (one numerical example for each fact). Mathematically curious ones are encouraged to prove them. 

Note a unit triangular matrix is a triangular matrix with all diagonal entries being 1.

1. The product of two upper (lower) triangular matrices is upper (lower) triangular.

2. The inverse of an upper (lower) triangular matrix is upper (lower) triangular.

3. The product of two unit upper (lower) triangular matrices is unit upper (lower) triangular.

4. The inverse of a unit upper (lower) triangular matrix is unit upper (lower) triangular.

5. An orthogonal upper (lower) triangular matrix is diagonal. (You just need to prove this.)

6. The product of two orthogonal matrices is orthogonal.